In [ ]:
%matplotlib widget

import numpy as np
import requests
import gzip
import os
import hashlib
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import Markdown

# Clasificación de dígitos

Utilizamos la base de datos de dígitos escritos a mano [MNIST](http://yann.lecun.com/exdb/mnist/).

In [ ]:
def fetch(url):
    hash = hashlib.md5(url.encode("utf-8")).hexdigest()
    path = os.path.join(".", hash)
    if os.path.isfile(path):
        with open(path, "rb") as f:
            data = f.read()
    else:
        with open(path, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(
        gzip.decompress(data),
        dtype=np.uint8,
    ).copy()

In [ ]:
image_data = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")
label_data = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")

## Formato de archivos

Los primeros 4 bytes del archivo de imágenes contienen el número mágico 2051

In [ ]:
[image_magic] = image_data[0:4][::-1].copy().view(np.uint32)
assert image_magic == 2051

Los siguientes 4 bytes del archivo de imágenes contienen la cantidad de ejemplos

In [ ]:
[image_total] = image_data[4:8][::-1].copy().view(np.uint32)

Los siguientes 4 bytes contienen la cantidad de renglones y los 4 bytes que le siguen la cantidad de columnas de cada imagen

In [ ]:
[image_rows, image_cols] = image_data[8:16][::-1].copy().view(np.uint32)

El resto de los bytes corresponden al valor en escala de grises de cada pixel, donde 0 significa blanco y 255 significa negro

In [ ]:
assert len(image_data[16:]) == image_total * image_rows * image_cols
images = image_data[16:].reshape((image_total, image_rows, image_cols))

Los primeros 4 bytes del archivo de etiquetas contienen el número mágico 2049

In [ ]:
[label_magic] = label_data[0:4][::-1].copy().view(np.uint32)
assert label_magic == 2049

Los siguientes 4 bytes del archivo de etiquetas contienen la cantidad de ejemplos

In [ ]:
[label_total] = label_data[4:8][::-1].copy().view(np.uint32)
assert label_total == image_total

El resto de los bytes corresponden a el dígito representado en cada imagen de ejemplo

In [ ]:
assert len(label_data[8:]) == label_total
labels = label_data[8:]

## Visualizando dígitos

In [ ]:
plt.ioff()
def show_example(n):
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.imshow(images[n], cmap=mpl.cm.gray_r)
    ax.set_title(f"La imagen contiene el dígito {labels[n]}")
    display(fig)

In [ ]:
show_example(3)

## Construcción del conjunto de entrenamiento

Cada ejemplo consiste de la pareja entrada-salida con las entradas siendo vectores unidimensionales con componentes enteras en el rango $[0,255]$ y las salidas siendo +1 si el dígito es cero y -1 en otro caso.

In [ ]:
Dtrain = [
    (images[i].reshape(image_rows * image_cols),
     +1 if labels[i] == 0 else -1)
    for i in range(image_total)
]

### Implementación de clasificación lineal con descenso de gradiente estocástico

In [ ]:
def features(x):
    return x

In [ ]:
def zero_weights():
    return np.zeros(image_rows * image_cols)

In [ ]:
def predict(w, x):
    return np.sign(w.dot(features(x)))

In [ ]:
def margin(x, y, w):
    return predict(w, x)*y

In [ ]:
def loss_hinge(x, y, w):
    return max(1 - margin(x, y, w), 0)

In [ ]:
def grad_loss_hinge(x, y, w):
    return -features(x)*y if margin(x, y, w) < 1 else np.zeros_like(w)

In [ ]:
def train_loss(Dtrain, loss, w):
    examples = len(Dtrain)
    total = sum(loss(x, y, w) for x, y in Dtrain)
    return total / examples

In [ ]:
def grad_train_loss(Dtrain, grad_loss, w):
    examples = len(Dtrain)
    total = sum(grad_loss(x, y, w) for x, y in Dtrain)
    return total / examples

In [ ]:
def eta_decreasing(init_eta):
    def decreaser(n):
        return init_eta / np.sqrt(n)
    return decreaser

In [ ]:
def GD(data, loss, grad_loss, init_weights, eta, epochs):
    w_hist = []
    tl_hist = []
    w = init_weights()
    for t in range(1, epochs + 1):
        tl = train_loss(data, loss, w)
        gtl = grad_train_loss(data, grad_loss, w)
        w_hist.append(w)
        tl_hist.append(tl)
        w = w - eta * gtl
    return w, (w_hist, tl_hist)

In [ ]:
def SGD(data, loss, grad_loss, init_weights, eta_func, epochs):
    w_hist = []
    tl_hist = []
    w = init_weights()
    n = 1
    for t in range(1, epochs + 1):
        w_hist.append(w)
        tl_hist.append(train_loss(data, loss, w))
        for x, y in data:
            gl = grad_loss(x, y, w)
            eta = eta_func(n)
            w = w - eta * gl
            n += 1
    return w, (w_hist, tl_hist)

In [ ]:
def SGDmb(data, loss, grad_loss, init_weights, eta_func, epochs, minibatches):
    w_hist = []
    tl_hist = []
    w = init_weights()
    n = 1
    for t in range(1, epochs + 1):
        w_hist.append(w)
        tl_hist.append(train_loss(data, loss, w))
        for i in range(0, len(data), minibatches):
            subdata = data[i:min(i + minibatches, len(data))]
            gtl = grad_train_loss(subdata, grad_loss, w)
            eta = eta_func(n)
            w = w - eta * gtl
            n += 1
    return w, (w_hist, tl_hist)

In [ ]:
%%time

gd_w, (gd_ws, gd_tls) = GD(
            data = D,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
             eta = 0.1,
          epochs = 10,
)

In [ ]:
%%time

sgd_w, (sgd_ws, sgd_tls) = SGD(
            data = D,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
        eta_func = eta_decreasing(1),
          epochs = 10,
)

In [ ]:
%%time

sgdmb_w, (sgdmb_ws, sgdmb_tls) = SGDmb(
            data = D,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
        eta_func = eta_decreasing(1),
          epochs = 10,
     minibatches = 100,
)

In [ ]:
plt.close()
plt.ioff()

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot()

epochs = list(range(1, len(sgd_tls) + 1))
ax.plot(epochs, gd_tls, label = "GD")
ax.plot(epochs, sgd_tls, label = "SGD")
ax.plot(epochs, sgdmb_tls, label = "SGDmb")
ax.legend()
ax.set_title("Learning with linear classifier and hinge loss")
ax.set_xlabel("epoch")
ax.set_ylabel("TrainLoss")
ax.set_xlim((epochs[0], epochs[-1]))

fig.canvas.header_visible = False
display(fig.canvas);